In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import sys
import os

sys.path.append('../tools')
import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
import sklearn
import random

import torchvision.transforms as T
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks
from models.rns_dataloader import get_data, get_data_by_episode

import data_utility
import annotation_utility
import interactive_plot

import warnings

warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argument*")
warnings.filterwarnings("ignore", ".*Set a lower value for log_every_n_steps if you want to see logs for the training epoch*")
warnings.filterwarnings("ignore", ".*exists and is not empty*")
warnings.filterwarnings("ignore", ".*Checkpoint directory {dirpath} exists and is not empty*")


In [3]:
data_dir = "../../../user_data/"
log_folder_root = '../../../user_data/logs/'
ckpt_folder_root = '../../../user_data/checkpoints/'

random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    # True ensures the algorithm selected by CUFA is deterministic
    torch.backends.cudnn.deterministic = True
    # torch.set_deterministic(True)
    # False ensures CUDA select the same algorithm each time the application is run
    torch.backends.cudnn.benchmark = False

import pytorch_lightning

pytorch_lightning.utilities.seed.seed_everything(seed=random_seed, workers=True)

Global seed set to 42


42

In [4]:
raw_annotations = pd.read_csv(data_dir + 'full_updated_anns_annotTbl_cleaned.csv')
ids = list(np.unique(raw_annotations[raw_annotations['descriptions'].notnull()]['HUP_ID']))
# ids = list(np.unique(raw_annotations['HUP_ID']))
ids

['HUP047',
 'HUP084',
 'HUP096',
 'HUP109',
 'HUP121',
 'HUP129',
 'HUP131',
 'HUP137',
 'HUP147',
 'HUP153',
 'HUP156',
 'HUP159',
 'HUP182',
 'HUP197',
 'HUP199',
 'HUP205',
 'RNS026',
 'RNS029']

In [21]:
data_import = data_utility.read_files(path=data_dir+'rns_data', patientIDs=ids,
                                      verbose=True)  # Import data with annotation

100%|██████████| 18/18 [00:33<00:00,  1.84s/it]


In [22]:
annotations = annotation_utility.read_annotation(annotation_path=data_dir +'full_updated_anns_annotTbl_cleaned.csv',
                                                 data=data_import, n_class=3)

C:\Users\Patrick Xu\Desktop\RNS_Annotation-Pipeline\scripts\RNS_LITT_ANNOTATION_PIPELINE\rns_scripts\../tools\annotation_utility.py:137: RuntimeWarning: invalid value encountered in cast
  temp_df['Episode_Index'] = episode_ind.astype(int)
C:\Users\Patrick Xu\Desktop\RNS_Annotation-Pipeline\scripts\RNS_LITT_ANNOTATION_PIPELINE\rns_scripts\../tools\annotation_utility.py:138: RuntimeWarning: invalid value encountered in cast
  temp_df['Episode_Start_Index'] = episode_start_ind.astype(int)
C:\Users\Patrick Xu\Desktop\RNS_Annotation-Pipeline\scripts\RNS_LITT_ANNOTATION_PIPELINE\rns_scripts\../tools\annotation_utility.py:139: RuntimeWarning: invalid value encountered in cast
  temp_df['Episode_End_Index'] = episode_end_ind.astype(int)
C:\Users\Patrick Xu\Desktop\RNS_Annotation-Pipeline\scripts\RNS_LITT_ANNOTATION_PIPELINE\rns_scripts\../tools\annotation_utility.py:137: RuntimeWarning: invalid value encountered in cast
  temp_df['Episode_Index'] = episode_ind.astype(int)


In [23]:
import numpy.lib.recfunctions as rfn
np.random.seed(seed=42)
annot = annotations.annotations
annot_nonseizure = annot[annot['Class_Code'] == 0]
annot_seizure = annot[annot['Class_Code'] == 1]
patient_list = list(np.unique(annot['Patient_ID']))
# patient_list = ['RNS026', 'HUP159', 'HUP129', 'HUP096', 'HUP182']
clip_dict = {}
for p in patient_list:
    seizure_start_index = np.array([])
    seizure_end_index = np.array([])
    nonseizure_start_index = np.array([])
    nonseizure_end_index = np.array([])
    global_episode_index_seizure = np.array([])
    global_episode_index_nonseizure = np.array([])
    start_index = annot_seizure[annot_seizure['Patient_ID'] == p]['Episode_Start_Index']
    end_index = annot_seizure[annot_seizure['Patient_ID'] == p]['Episode_End_Index']
    annot_start_list = annot_seizure[annot_seizure['Patient_ID'] == p]['Annotation_Start_Index']
    annot_end_list = annot_seizure[annot_seizure['Patient_ID'] == p]['Annotation_End_Index']
    episode_index = annot_seizure[annot_seizure['Patient_ID'] == p]['Episode_Index']

    for i, slel in enumerate(zip(annot_start_list, annot_end_list, episode_index.index)):
        sl_order = np.argsort(slel[0])
        sl = np.array(slel[0])[sl_order]
        el = np.array(slel[1])[sl_order]
        ei = slel[2]

        annot_array = np.vstack((sl, el))
        seizure_start_index = np.hstack((seizure_start_index, annot_array[0, :]))
        seizure_end_index = np.hstack((seizure_end_index, annot_array[1, :]))

        nonseizure_start_index = np.hstack((nonseizure_start_index, start_index.iloc[i]))
        nonseizure_end_index = np.hstack((nonseizure_end_index, annot_array[0, 0]))

        nonseizure_start_index = np.hstack((nonseizure_start_index, annot_array[1, -1]))
        nonseizure_end_index = np.hstack((nonseizure_end_index, end_index.iloc[i]))

        if annot_array.shape[1] > 1:
            nonseizure_start_index = np.hstack((nonseizure_start_index,annot_array[1, :-1]))
            nonseizure_end_index = np.hstack((nonseizure_end_index,  annot_array[0, 1:]))


        global_episode_index_seizure = np.hstack((global_episode_index_seizure,
                                                  np.repeat(ei, len(seizure_start_index) -
                                                            len(global_episode_index_seizure))))
        global_episode_index_nonseizure = np.hstack((global_episode_index_nonseizure,
                                                     np.repeat(ei, len(nonseizure_start_index) -
                                                               len(global_episode_index_nonseizure))))

    assert len(global_episode_index_nonseizure) == len(nonseizure_start_index)
    assert len(global_episode_index_seizure) == len(seizure_start_index)

    start_index = annot_nonseizure[annot_nonseizure['Patient_ID'] == p]['Episode_Start_Index']
    end_index = annot_nonseizure[annot_nonseizure['Patient_ID'] == p]['Episode_End_Index']
    episode_index = start_index.index

    nonseizure_ind_arr = np.vstack(
        [nonseizure_start_index,
         nonseizure_end_index,
         global_episode_index_nonseizure]).astype(int)

    seizure_ind_arr = np.vstack(
        [seizure_start_index,
         seizure_end_index,
         global_episode_index_seizure]).astype(int)

    nonseizure_ind_arr_eps = np.vstack(
        [start_index,
         end_index,
         episode_index]).astype(int)

    nonseizure_clip_temp = np.hstack((nonseizure_ind_arr, nonseizure_ind_arr_eps))
    nonseizure_clip_label = np.zeros(nonseizure_clip_temp.shape[1]).astype(int)
    non_seizure_clip = np.vstack((nonseizure_clip_temp, nonseizure_clip_label))

    seizure_clip_temp = np.vstack(
            [seizure_start_index,
             seizure_end_index,
             global_episode_index_seizure]).astype(int)
    seizure_clip_label = np.ones(seizure_clip_temp.shape[1]).astype(int)
    seizure_clip = np.vstack((seizure_clip_temp, seizure_clip_label))

    combined_clip = np.hstack((seizure_clip, non_seizure_clip))

    valid = np.where((combined_clip[1] - combined_clip[0]) > 500)

    combined_clip = combined_clip[:,valid].squeeze()

    if combined_clip.shape[1]>0:
        # shuffled_index = np.arange(combined_clip.shape[1])
        # np.random.shuffle(shuffled_index)
        # clip_dict[p] = combined_clip[:, shuffled_index]


        structured_array = rfn.unstructured_to_structured(combined_clip.T.astype(int),
                                      np.dtype(
                                          [('start_index', 'int32'), ('end_index', 'int32'), ('episode_index', 'int32'),
                                           ('label', 'int32')]))

        clip_dict[p] = combined_clip.T[np.argsort(structured_array, order=['episode_index','start_index'])].T

np.save(data_dir + 'rns_test_cache/clip_dict.npy', clip_dict)

In [148]:
import numpy.lib.recfunctions as rfn

np.random.seed(seed=42)
annot = annotations.annotations
annot_nonseizure = annot[annot['Class_Code'] == 0]
annot_seizure = annot[annot['Class_Code'] == 1]
patient_list = ['HUP047',
       'HUP059',
       'HUP084',
       'HUP096',
       'HUP101',
       'HUP108',
       'HUP109',
       'HUP121',
       'HUP127',
       'HUP128',
       'HUP129',
       'HUP131',
       'HUP136',
       # 'HUP137',
       'HUP143',
       'HUP147',
       # 'HUP153',
       'HUP156',
       'HUP159',
       'HUP182',
       'HUP192',
       'HUP197',
       'HUP199',
       'HUP205',
       'RNS021',
       'RNS022',
       'RNS026',
       'RNS029']
# patient_list = ['RNS026', 'HUP159', 'HUP129', 'HUP096', 'HUP182']
clip_dict = {}
# for p in patient_list:
for p in patient_list:
    # print(p)
    seizure_start_index = np.array([])
    seizure_end_index = np.array([])
    nonseizure_start_index = np.array([])
    nonseizure_end_index = np.array([])
    global_episode_index_seizure = np.array([])
    global_episode_index_nonseizure = np.array([])

    annotation_list = []

    start_index = annot[annot['Patient_ID'] == p]['Episode_Start_Index']
    end_index = annot[annot['Patient_ID'] == p]['Episode_End_Index']
    annot_start_list = annot[annot['Patient_ID'] == p]['Annotation_Start_Index']
    annot_end_list = annot[annot['Patient_ID'] == p]['Annotation_End_Index']
    j = 0
    for i in range(len(start_index)):
        initial_arr = np.zeros(end_index.iloc[i] - start_index.iloc[i])
        if len(annot_start_list.iloc[i]) > 0:
            sl_order = np.argsort(annot_start_list.iloc[i])
            sl = np.array(annot_start_list.iloc[i])[sl_order]
            el = np.array(annot_end_list.iloc[i])[sl_order]

            for si, ei in zip(sl, el):
                initial_arr[si - start_index.iloc[i]:ei - start_index.iloc[i]] = 1

        annotation_list.append(initial_arr)

    ind_arr = np.vstack(
        [start_index,
         end_index,
         start_index.index]).astype(int)

    valid = np.where((ind_arr[1] - ind_arr[0]) > 500)
    combined_clip = ind_arr[:, valid].squeeze()
    annotation_list = np.array(annotation_list, dtype=object)[valid]
    combined_clip = np.vstack((combined_clip, annotation_list))

    if combined_clip.shape[1]>0:

        clip_dict[p] = combined_clip

np.save(data_dir + 'rns_test_cache/clip_dict.npy', clip_dict)

array([[995590, 1329499, 2328305, 3584137, 4102460, 4177243, 4850221,
        6379914, 7127405, 7352695, 8238876, 8734496, 9478014, 9658246,
        9816002, 11474261, 12082549, 12510541, 17604018, 17773125,
        20327009, 20372052, 20877529, 22755604, 23402984, 23785908,
        25182544, 30470712, 31101419, 38707538, 39360873, 40262120,
        40307172, 40374725, 47095918, 49371398, 54631255, 55531462,
        59361792, 63012791, 72118005, 73019502, 77797587, 81787042,
        87286122, 87376250, 90592854, 92801678],
       [1018115, 1352025, 2350838, 3606654, 4120950, 4199768, 4872752,
        6402432, 7149927, 7375219, 8261404, 8757023, 9500533, 9680771,
        9838525, 11496790, 12105067, 12533065, 17626525, 17795653,
        20349528, 20394566, 20900049, 22778124, 23425506, 23808438,
        25205066, 30493229, 31123953, 38730076, 39383397, 40284629,
        40329665, 40397254, 47118431, 49393927, 54653767, 55553974,
        59384333, 63035308, 72140518, 73042043, 77820128, 

In [124]:
annotation_list.T.shape

(96,)

In [79]:
annot_start_list

1910                    []
1911                    []
1912             [2335547]
1913                    []
1914                    []
1915                    []
1916                    []
1917                    []
1918                    []
1919    [7354774, 7360415]
1920                    []
1921             [8746040]
1922                    []
1923             [9665517]
1925             [9823226]
1926                    []
1927                    []
1928            [12517826]
1929                    []
1930            [17780466]
1932            [20328635]
1933                    []
1934                    []
1935                    []
1936                    []
1937                    []
1938                    []
1939                    []
1940                    []
1941                    []
1942                    []
1943            [40262773]
1944            [40310001]
1945                    []
1946            [47098297]
1947                    []
1948            [54633602]
1

In [191]:
from scipy.stats import mode

window_len = 1
stride = 1
concat_n = 4
for id in clip_dict.keys():
    data_import[id].set_window_parameter(window_length=window_len, window_displacement=stride)
    data_import[id].set_concatenation_parameter(concatenate_window_n=concat_n)
    window_indices, _ = data_import[id].get_windowed_data(clip_dict[id][0], clip_dict[id][1])
    import_indices = []
    import_label = []
    import_clip_indices = []
    import_start_indicies = []
    import_patient_ID = []
    for i, ind in enumerate(window_indices):
        indices = window_indices[i]+1-clip_dict[id][0][i]
        offsets = np.arange(249)
        full_indices = indices[:,0][:, np.newaxis] + offsets
        slices_no_loop = clip_dict[id][3][i][full_indices]
        mode_result = mode(slices_no_loop, axis=1)
        mode_values = mode_result.mode

        # print(mode_values)
        import_label.append(mode_values)
        import_indices.append(np.repeat(clip_dict[id][2][i], len(ind)))
        import_clip_indices.append(np.arange(len(ind)))
        import_start_indicies.append(np.repeat(clip_dict[id][0][i], len(ind)))
        import_patient_ID.append(np.repeat(id, len(ind)))

    import_label = np.hstack(import_label)
    import_indices = np.hstack(import_indices)
    import_clip_indices = np.hstack(import_clip_indices)
    import_start_indicies = np.hstack(import_start_indicies)
    import_patient_ID = np.hstack(import_patient_ID)

    data_import[id].normalize_windowed_data()
    _, concatenated_data = data_import[id].get_concatenated_data(data_import[id].windowed_data, arrange='channel_stack')

    assert np.hstack(import_label).shape[0] == concatenated_data.shape[0]

    np.save(data_dir+'rns_test_cache/' + id + '.npy', {'data': concatenated_data, 'label': import_label, 'patientID': import_patient_ID, 'indices': np.vstack([import_indices,import_clip_indices,import_start_indicies]).T})

(8434,) (8434,) (8434,)
(4302,) (4302,) (4302,)
(8769,) (8769,) (8769,)
(4223,) (4223,) (4223,)
(6815,) (6815,) (6815,)
(6865,) (6865,) (6865,)
(5824,) (5824,) (5824,)
(6389,) (6389,) (6389,)
(8612,) (8612,) (8612,)
(12906,) (12906,) (12906,)
(3769,) (3769,) (3769,)


C:\Users\Patrick Xu\Desktop\RNS_Annotation-Pipeline\scripts\RNS_LITT_ANNOTATION_PIPELINE\rns_scripts\../tools\segmentation.py:113: RuntimeWarning: invalid value encountered in cast
  concatenate_ind_table = np.empty((clip_len_cumsum[-1], concat_n * 2 + 1)).astype(int)


(2313,) (2313,) (2313,)
(4483,) (4483,) (4483,)
(8561,) (8561,) (8561,)
(8688,) (8688,) (8688,)


In [8]:
from models.rns_dataloader import RNS_Downstream
from models.SwaV import SwaV

In [9]:
import torch
import torchvision
from torch import nn

from lightly.data import LightlyDataset, SwaVCollateFunction
from lightly.loss import SwaVLoss
from lightly.loss.memory_bank import MemoryBankModule
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes


In [10]:
def collate_fn(batch):
    info = list(zip(*batch))
    data = info[0]
    label = info[1]
    return torch.stack(data), torch.stack(label)

In [4]:
data_list = os.listdir(data_dir+'rns_test_cache')[1:]

# data_list = ['HUP182.npy',   'HUP129.npy',   'HUP109.npy', 'HUP156.npy', 'HUP096.npy', 'RNS026.npy',  'HUP159.npy']
# data_list = ['RNS026.npy', 'HUP159.npy', 'HUP129.npy', 'HUP096.npy', 'HUP182.npy']
train_data, train_label, test_data, test_label, train_index, test_index = get_data_by_episode(data_list, split=0.8)
# data, label,_,_ = get_data(data_list, split=1)
# train_data, test_data, train_label, test_label = sklearn.model_selection.train_test_split(data, label, test_size=0.8, random_state=42)

print(train_data.shape)
print(train_label.shape)
print(train_index.shape)
print(test_data.shape)
print(test_label.shape)
print(test_index.shape)

16it [00:05,  3.09it/s]

(992,)
(992,)
(992,)
(270,)
(270,)
(270,)


In [5]:
from models.SupervisedDownstream import SupervisedDownstream

In [6]:
swav = SwaV().load_from_checkpoint(ckpt_folder_root + 'rns_swav_50_12/rns_swav-epoch=82-swav_loss=2.58204.ckpt')
model = SupervisedDownstream(swav.backbone, swav.projection_head)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

checkpoint_callback = pl_callbacks.ModelCheckpoint(monitor='val_loss',
                                                   filename='rns_swav_50_all_linear_eval-{epoch:02d}-{val_loss:.5f}', save_last=True, save_top_k=-1, dirpath=ckpt_folder_root + 'rns_swav_50_all_linear_eval')
csv_logger = pl_loggers.CSVLogger(log_folder_root, name='rns_swav_50_all_linear_eval')

trainer = pl.Trainer(logger=csv_logger, max_epochs=80, callbacks=[checkpoint_callback], accelerator='gpu', devices=1,precision=16)

NameError: name 'SwaV' is not defined

In [14]:
from models.rns_dataloader import *
train_dataset = RNS_Active_by_episode_LSTM(train_data, train_label, transform=True, astensor=True)
test_dataset = RNS_Active_by_episode_LSTM(test_data, test_label, transform=False, astensor=True)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)

val_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=False,
    drop_last=True,
)

trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                       | Type               | Params
------------------------------------------------------------------
0 | backbone                   | Sequential         | 23.5 M
1 | fc1                        | Linear             | 1.0 M 
2 | fc2                        | Linear             | 32.8 K
3 | dp                         | Dropout1d          | 0     
4 | fc3                        | Linear             | 520   
5 | fc4                        | Linear             | 18    
6 | softmax                    | Softmax            | 0     
7 | unfreeze_backbone_at_epoch | SwaVProjectionHead | 4.5 M 
------------------------------------------------------------------
29.1 M    Trainable params
0         Non-trainable params
29.1 M    Total params
58.102    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

ValueError: too many values to unpack (expected 2)

In [16]:
test_dataset = RNS_Downstream(test_data, test_label, transform=False, astensor=True)
val_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=False,
    drop_last=True,
)

data loaded
(21556, 249, 36)
(21556,)


In [17]:
data_list = os.listdir(data_dir+'rns_test_cache')[1:]
# data_list = ['RNS026_seizure.npy', 'HUP159_seizure.npy', 'HUP129_seizure.npy', 'HUP096_seizure.npy','RNS026_nonseizure.npy', 'HUP159_nonseizure.npy', 'HUP129_nonseizure.npy', 'HUP096_nonseizure.npy']
class RNS_Downstream(Dataset):
    def __init__(self, file_names, transform=True, astensor=True):
        self.file_names = file_names
        self.transform = transform

        file_name_temp = self.file_names[0]
        cache = np.load(data_dir +'rns_test_cache/' + file_name_temp, allow_pickle=True)
        temp_file = cache.item().get('data')

        self.data = np.empty((0, temp_file.shape[1], temp_file.shape[2]))
        self.label = np.array([])
        print(self.data.shape)

        for name in tqdm(self.file_names):
            cache = np.load(data_dir +'rns_test_cache/' + name, allow_pickle=True)
            data = cache.item().get('data')
            label = cache.item().get('label')
            self.data = np.vstack((self.data, data))
            self.label = np.hstack((self.label, label))

        print('data loaded')

        self.label = self.label[np.newaxis].T

        self.length = len(self.data)

        if astensor:
            self.augmentation = T.Compose([
                T.ToPILImage(),
                T.Resize((256, 256), interpolation=T.InterpolationMode.NEAREST),
                T.RandomApply([T.ColorJitter()], p=0.5),
                T.RandomApply([T.GaussianBlur(kernel_size=(3, 3))], p=0.5),
                T.RandomInvert(p=0.2),
                T.RandomPosterize(4, p=0.2),
                T.ToTensor()
            ])

            self.totensor = T.Compose([
                T.ToPILImage(),
                T.Resize((256, 256), interpolation=T.InterpolationMode.NEAREST),
                T.ToTensor()
            ])
        else:
            self.augmentation = T.Compose([
                T.ToPILImage(),
                T.Resize((256, 256), interpolation=T.InterpolationMode.NEAREST),
                T.RandomApply([T.ColorJitter()], p=0.5),
                T.RandomApply([T.GaussianBlur(kernel_size=(3, 3))], p=0.5),
                T.RandomInvert(p=0.2),
                T.RandomPosterize(4, p=0.2),
            ])

            self.totensor = T.Compose([
                T.ToPILImage(),
                T.Resize((256, 256), interpolation=T.InterpolationMode.NEAREST),
            ])


    def __len__(self):
        return self.length


    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]

        if self.transform:
            concat_len = data.shape[1] / 4
            channel_index = np.arange(4)
            np.random.shuffle(channel_index)
            channel_index = channel_index * concat_len + (concat_len - 1) / 2
            channel_index = np.repeat(channel_index, concat_len)
            concate_len_1 = (concat_len - 1) / 2
            a_repeat = np.arange(-concate_len_1, concate_len_1 + 1)[np.newaxis].T
            base_repeat = np.repeat(a_repeat, 4, axis=1).T.flatten()
            channel_index = channel_index + base_repeat
            data = data[channel_index.astype(int)]
            data = torch.from_numpy(data).clone()
            data = data.repeat(3, 1, 1)
            data = self.augmentation(data)

        else:
            concat_len = data.shape[1] / 4
            channel_index = np.arange(4)
            # np.random.shuffle(channel_index)
            channel_index = channel_index * concat_len + (concat_len - 1) / 2
            channel_index = np.repeat(channel_index, concat_len)
            concate_len_1 = (concat_len - 1) / 2
            a_repeat = np.arange(-concate_len_1, concate_len_1 + 1)[np.newaxis].T
            base_repeat = np.repeat(a_repeat, 4, axis=1).T.flatten()
            channel_index = channel_index + base_repeat
            data = data[channel_index.astype(int)]
            data = torch.from_numpy(data).clone()
            data = data.repeat(3, 1, 1)
            data = self.totensor(data)

        return data, torch.from_numpy(label).to(dtype=torch.long), None

dataset = RNS_Downstream(data_list, transform=False,astensor = True)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=False,
    drop_last=True,
)

(0, 249, 36)


100%|██████████| 15/15 [00:12<00:00,  1.21it/s]

data loaded


In [48]:
predictions = trainer.predict(model,val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 472it [00:00, ?it/s]

In [49]:
output_list = []
target_list = []
emb_list = []
m = nn.Softmax(dim=1)
for pred, y, emb in predictions:
    output_list.append(pred)
    target_list.append(y)
    emb_list.append(emb)

In [50]:
pred_raw = torch.vstack(output_list)
target = torch.vstack(target_list)
emb = torch.vstack(emb_list)
out = torch.argmax(pred_raw, dim=1)

In [42]:
m(pred_raw.float())[:,0]

tensor([0.9998, 0.9998, 0.9998,  ..., 0.9928, 0.9836, 0.9926])

In [43]:
len(list(np.unique(test_index)))

21530

In [23]:
index_list = []
for i in range(len(list(np.unique(test_index['episode_index'])))):

    print(i)

    index_location = np.where(test_index['episode_index'] == list(np.unique(test_index['episode_index']))[i])[0]

    # print(target[index_location[np.argsort(test_index[index_location], order=['start_index','slice_index'])]].squeeze(1))
    # print(out[index_location[np.argsort(test_index[index_location], order=['start_index','slice_index'])]])
    index_array = test_index[index_location[np.argsort(test_index[index_location], order=['start_index','slice_index'])]]
    start_ind = np.unique(index_array['start_index'])

    annot = annotations.annotations[annotations.annotations.index == np.unique(index_array['episode_index'])[0]]

    start_indexes = []
    start_indexes.extend(annot.Annotation_Start_Index.item())
    start_indexes.extend(annot.Annotation_End_Index.item())
    start_indexes.append(annot.Episode_Start_Index.item())
    start_indexes.append(annot.Episode_End_Index.item())
    # print(start_indexes)

    in_list = []
    for si in start_ind:
        in_list.append(si in start_indexes)

    assert True in in_list

# [annot.Annotation_Start_Index.item(), annot.Episode_Start_Index, ]

    # # print(annotations.annotations[annotations.annotations.index == slice_index])
    # if class_code == 0:
    #     try:
    #         assert (((start_index == annot.Episode_Start_Index) | (end_index == annot.Episode_End_Index)).item())
    #     except:
    #         print(ind)
    #         print(annot)
    #
    # elif class_code == 1:
    #     assert start_index in annot.Annotation_Start_Index.item()

0


NameError: name 'annotations' is not defined

In [51]:
sklearn.metrics.accuracy_score(torch.argmax(pred_raw, dim=1), target)

0.917981072555205

In [52]:
clf_report = sklearn.metrics.classification_report(torch.argmax(pred_raw, dim=1), target, digits=6)

print(f"Classification Report : \n{clf_report}")

Classification Report : 
              precision    recall  f1-score   support

           0   0.931611  0.946978  0.939231     14428
           1   0.888970  0.859287  0.873876      7128

    accuracy                       0.917981     21556
   macro avg   0.910290  0.903133  0.906554     21556
weighted avg   0.917510  0.917981  0.917620     21556



In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [ ]:
len(emb)

In [ ]:
pca_comp_n = 30
batch_size = 32

pca = PCA(n_components=pca_comp_n, copy=True).fit(emb)
p = pca.transform(emb)

# ind = np.random.choice(len(emb), 10000)
#
tsne = TSNE(n_components=2, verbose=1, perplexity=75, random_state=142, init='pca')
z = tsne.fit_transform(emb)
interictal_inds = np.where(target == 0)[0]
ictal_inds = np.where(target == 1)[0]

In [ ]:
pca.explained_variance_ratio_

In [ ]:
len(interictal_inds)

In [ ]:
len(ictal_inds)

In [ ]:
# spc = p
#
# fig = plt.figure(figsize=(10, 8))
# ax = fig.add_subplot(projection='3d')
# ax.scatter(spc[interictal_inds,0],spc[interictal_inds,1],spc[interictal_inds,2],c='gold',label= 'interictal')
# ax.scatter(spc[ictal_inds, 0], spc[ictal_inds, 1],spc[ictal_inds, 2], c='royalblue', label='ictal')
# # plt.title('Swav Embedding t-SNE')
# ax.set_xlabel('comp 1')
# ax.set_ylabel("comp 2")
# ax.set_zlabel("comp 2")
# plt.legend()
# # plt.xlim(-67, 74)
# # plt.ylim(-67, 75)
# plt.grid()
# # plt.show()

In [ ]:
spc = p
plt.figure(figsize=(10, 8))
plt.scatter(spc[interictal_inds,0],spc[interictal_inds,1],c='gold',label= 'interictal')
plt.scatter(spc[ictal_inds, 0], spc[ictal_inds, 1], c='royalblue', label='ictal',s=0.5)
# plt.title('Swav Embedding t-SNE')
plt.xlabel('comp 1')
plt.ylabel("comp 2")
plt.legend()
# plt.xlim(-67, 74)
# plt.ylim(-67, 75)
plt.grid()
plt.show()

In [ ]:
# dt = np.vstack((z[:,0], z[:,1])).T
interactive_plot.interactive_plot(z, ['RNS026', 'HUP159', 'HUP129', 'HUP096'], data_import, color_override=target)

In [ ]:
interactive_plot.interactive_plot(z, ['HUP159'], data_import, color_override=target)

In [ ]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(m(pred_raw.float()), target, pos_label=1)
sklearn.metrics.auc(fpr, tpr)

In [ ]:
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

RocCurveDisplay.from_predictions(
    target,
    m(pred_raw.float())[:,1],
    color="darkorange",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.hlines(y = 1, xmin = 0, xmax = 1)
plt.vlines(x = 0, ymin = 0, ymax = 1)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()

In [ ]:
PrecisionRecallDisplay.from_predictions(
    target,
    m(pred_raw.float())[:,1],
    color="darkorange",
)
# plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")

plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(target,
    m(pred_raw.float())[:,1])
metrics.auc(fpr, tpr)

In [ ]:
output[:, 1]

In [ ]:

output = torch.argmax(output, dim=1)
output = output.detach().cpu().numpy()
target = target.squeeze().detach().cpu().numpy()

In [ ]:
import sklearn

clf_report = sklearn.metrics.classification_report(output, target, digits=6)

print(f"Classification Report : \n{clf_report}")

In [ ]:
for batch, label in tqdm(val_dataloader):
    batch = batch.to(device)
    label = label.to(device)
    label = F.one_hot(label).squeeze()
    outputs = model(batch)
    print(batch)
    loss = sigmoid_focal_loss(pred.float(), label.float(), alpha=0.5, gamma=8, reduction='mean')
    print(loss)
    break

In [ ]:
# import copy
# import torch
# import torchvision
# from torch import nn
#
# from lightly.data import DINOCollateFunction, LightlyDataset
# from lightly.loss import DINOLoss
# from lightly.models.modules import DINOProjectionHead
# from lightly.models.utils import deactivate_requires_grad, update_momentum
# from lightly.utils.scheduler import cosine_schedule
#
#
# class DINO(torch.nn.Module):
#     def __init__(self, backbone, input_dim):
#         super().__init__()
#         self.student_backbone = backbone
#         self.student_head = DINOProjectionHead(
#             input_dim, 512, 64, 2048, freeze_last_layer=1
#         )
#         self.teacher_backbone = copy.deepcopy(backbone)
#         self.teacher_head = DINOProjectionHead(input_dim, 512, 64, 2048)
#         deactivate_requires_grad(self.teacher_backbone)
#         deactivate_requires_grad(self.teacher_head)
#
#     def forward(self, x):
#         y = self.student_backbone(x).flatten(start_dim=1)
#         z = self.student_head(y)
#         return z
#
#     def forward_teacher(self, x):
#         y = self.teacher_backbone(x).flatten(start_dim=1)
#         z = self.teacher_head(y)
#         return z
#
#
# resnet = torchvision.models.resnet18()
# backbone = nn.Sequential(*list(resnet.children())[:-1])
# input_dim = 512
# # instead of a resnet you can also use a vision transformer backbone as in the
# # original paper (you might have to reduce the batch size in this case):
# # backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vits16', pretrained=False)
# # input_dim = backbone.embed_dim
#
# model = DINO(backbone, input_dim)
#
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)
#
# # # we ignore object detection annotations by setting target_transform to return 0
# # pascal_voc = torchvision.datasets.VOCDetection(
# #     "datasets/pascal_voc", download=True, target_transform=lambda t: 0
# # )
# # dataset = LightlyDataset.from_torch_dataset(pascal_voc)
# # # or create a dataset from a folder containing images or videos:
# # # dataset = LightlyDataset("path/to/folder")
#
# collate_fn = DINOCollateFunction(solarization_prob = 0, hf_prob = 0,vf_prob = 0,rr_prob=0,cj_prob=0,random_gray_scale=0)
#
# dataloader = torch.utils.data.DataLoader(
#     train_set,
#     batch_size=64,
#     collate_fn=collate_fn,
#     shuffle=True,
#     drop_last=True,
#     num_workers=1,
# )
#
# criterion = DINOLoss(
#     output_dim=2048,
#     warmup_teacher_temp_epochs=5,
# )
# # move loss to correct device because it also contains parameters
# criterion = criterion.to(device)
#
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#
# epochs = 10
#
# print("Starting Training")
# for epoch in range(epochs):
#     total_loss = 0
#     momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
#     for views, _, _ in tqdm(dataloader):
#         update_momentum(model.student_backbone, model.teacher_backbone, m=momentum_val)
#         update_momentum(model.student_head, model.teacher_head, m=momentum_val)
#         views = [view.to(device) for view in views]
#         global_views = views[:2]
#         teacher_out = [model.forward_teacher(view) for view in global_views]
#         student_out = [model.forward(view) for view in views]
#         loss = criterion(teacher_out, student_out, epoch=epoch)
#         total_loss += loss.detach()
#         loss.backward()
#         # We only cancel gradients of student head.
#         model.student_head.cancel_last_layer_gradients(current_epoch=epoch)
#         optimizer.step()
#         optimizer.zero_grad()
#
#     avg_loss = total_loss / len(dataloader)
#     print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

In [ ]:
augmentation = T.Compose([
    T.ToPILImage(),
    T.Resize((256, 512), interpolation=T.InterpolationMode.NEAREST),
    T.RandomApply([T.ColorJitter()], p=0.5),
    T.RandomApply([T.GaussianBlur(kernel_size=(3, 3))], p=0.5),
    T.RandomInvert(p=0.2),
    T.RandomPosterize(4, p=0.2),
])

data = ictal_data_X[0]

channel_index = np.arange(data.shape[0])
np.random.shuffle(channel_index)
data = data[channel_index]
data = torch.from_numpy(data).clone()
data = data.repeat(3, 1, 1)
data = augmentation(data)
data

In [ ]:
channel_index

In [ ]:
data[channel_index]

In [ ]:
data

In [ ]:
#
# print("Starting Training")
# for epoch in range(50):
#     total_loss = 0
#     i = 0
#     for batch, label in tqdm(dataloader):
#         batch = batch.to(device)
#         # print(type(batch))
#         label = label.to(device)
#         label = F.one_hot(label).squeeze()
#         outputs = model(batch)
#         loss = sigmoid_focal_loss(outputs.float(),label.float(), alpha = 0.25, gamma = 7,reduction = 'mean')
#         total_loss += loss.detach()
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#
#     avg_loss = total_loss / len(dataloader)
#     torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': avg_loss,
#             }, 'ckpt/checkpoint'+str(epoch)+'.pth')
#
#     print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")